<a href="https://colab.research.google.com/github/sarahwierzbicki/projects/blob/main/job_set_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scipy

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import keras
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [ ]:
df = pd.read_csv('/content/jobpostings.csv')

In [ ]:
df

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713400e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712860e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713280e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712900e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713450e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,3906267117,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,195000.0,YEARLY,"Walnut Creek, CA",56120.0,1.0,NaN,...,NaN,1.713570e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,94595.0,6013.0
123845,3906267126,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,NaN,United States,1124131.0,3.0,NaN,...,NaN,1.713570e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN
123846,3906267131,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,NaN,"Spokane, WA",90552133.0,3.0,NaN,...,NaN,1.713570e+12,epsoperations.bamboohr.com,0,FULL_TIME,NaN,NaN,NaN,99201.0,53063.0
123847,3906267195,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,NaN,"Texas, United States",2793699.0,4.0,NaN,...,NaN,1.713570e+12,NaN,0,FULL_TIME,NaN,NaN,NaN,NaN,NaN


pre-processing

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [ ]:
#drop duplicates
df = df.drop_duplicates()

In [ ]:
#drop unneeded columns
df.drop('views', axis = 1, inplace = True)
df.drop('job_id', axis = 1, inplace = True)
df.drop('company_id', axis = 1, inplace = True)
df.drop('applies', axis = 1, inplace = True)
df.drop('original_listed_time', axis = 1, inplace = True)
df.drop('job_posting_url', axis = 1, inplace = True)
df.drop('application_url', axis = 1, inplace = True)
df.drop('application_type', axis = 1, inplace = True)
df.drop('expiry', axis = 1, inplace = True)
df.drop('closed_time', axis = 1, inplace = True)
df.drop('listed_time', axis = 1, inplace = True)
df.drop('posting_domain', axis = 1, inplace = True)
df.drop('sponsored', axis = 1, inplace = True)
df.drop('currency', axis = 1, inplace = True)
df.drop('fips', axis = 1, inplace = True)

In [ ]:
#drop med, min and max salary only use normalizaed
df.drop('max_salary', axis = 1, inplace = True)
df.drop('min_salary', axis = 1, inplace = True)
df.drop('med_salary', axis = 1, inplace = True)

In [ ]:
df

,company_name,title,description,pay_period,location,formatted_work_type,remote_allowed,formatted_experience_level,skills_desc,work_type,compensation_type,normalized_salary,zip_code
0,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,HOURLY,"Princeton, NJ",Full-time,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,FULL_TIME,BASE_SALARY,38480.0,8540.0
1,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",HOURLY,"Fort Collins, CO",Full-time,NaN,NaN,NaN,FULL_TIME,BASE_SALARY,83200.0,80521.0
2,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,YEARLY,"Cincinnati, OH",Full-time,NaN,NaN,We are currently accepting resumes for FOH - A...,FULL_TIME,BASE_SALARY,55000.0,45202.0
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,YEARLY,"New Hyde Park, NY",Full-time,NaN,NaN,This position requires a baseline understandin...,FULL_TIME,BASE_SALARY,157500.0,11040.0
4,NaN,Service Technician,Looking for HVAC service tech with experience ...,YEARLY,"Burlington, IA",Full-time,NaN,NaN,NaN,FULL_TIME,BASE_SALARY,70000.0,52601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,YEARLY,"Walnut Creek, CA",Full-time,NaN,Mid-Senior level,NaN,FULL_TIME,BASE_SALARY,157500.0,94595.0
123845,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,United States,Full-time,1.0,Mid-Senior level,NaN,FULL_TIME,NaN,NaN,NaN
123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,"Spokane, WA",Full-time,1.0,Mid-Senior level,NaN,FULL_TIME,NaN,NaN,99201.0
123847,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,"Texas, United States",Full-time,1.0,NaN,NaN,FULL_TIME,NaN,NaN,NaN


In [ ]:
!pip install tensorflow-data-validation
!pip install apache-beam
import tensorflow_data_validation as tfdv
import tensorflow as tf

In [ ]:
tfdv_jobs = tfdv.generate_statistics_from_dataframe(dataframe= df)
tfdv.visualize_statistics(tfdv_jobs)

cut out jobs with no experience level (~25%)
drop without zipcode
drop pay period
drop compensation type
remove skills description (too many NaN, hopefully extract from job description

In [ ]:
df

,company_name,title,description,pay_period,location,formatted_work_type,remote_allowed,formatted_experience_level,skills_desc,work_type,compensation_type,normalized_salary,zip_code
0,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,HOURLY,"Princeton, NJ",Full-time,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,FULL_TIME,BASE_SALARY,38480.0,8540.0
1,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",HOURLY,"Fort Collins, CO",Full-time,NaN,NaN,NaN,FULL_TIME,BASE_SALARY,83200.0,80521.0
2,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,YEARLY,"Cincinnati, OH",Full-time,NaN,NaN,We are currently accepting resumes for FOH - A...,FULL_TIME,BASE_SALARY,55000.0,45202.0
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,YEARLY,"New Hyde Park, NY",Full-time,NaN,NaN,This position requires a baseline understandin...,FULL_TIME,BASE_SALARY,157500.0,11040.0
4,NaN,Service Technician,Looking for HVAC service tech with experience ...,YEARLY,"Burlington, IA",Full-time,NaN,NaN,NaN,FULL_TIME,BASE_SALARY,70000.0,52601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,YEARLY,"Walnut Creek, CA",Full-time,NaN,Mid-Senior level,NaN,FULL_TIME,BASE_SALARY,157500.0,94595.0
123845,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,United States,Full-time,1.0,Mid-Senior level,NaN,FULL_TIME,NaN,NaN,NaN
123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,"Spokane, WA",Full-time,1.0,Mid-Senior level,NaN,FULL_TIME,NaN,NaN,99201.0
123847,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,"Texas, United States",Full-time,1.0,NaN,NaN,FULL_TIME,NaN,NaN,NaN


In [ ]:
df.drop('pay_period', axis = 1, inplace = True)
df.drop('compensation_type', axis = 1, inplace = True)
df.drop('skills_desc', axis = 1, inplace = True)
df.drop('remote_allowed', axis = 1, inplace = True)

In [ ]:
#drop rows with no zipcode (NaN) and no experience level
df = df.dropna(subset=['zip_code'])
df = df.dropna(subset=['formatted_experience_level'])

In [ ]:
df

,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code
70,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,NaN,46312.0
84,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,NaN,41048.0
85,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0
101,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0
102,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,NaN,97114.0
...,...,...,...,...,...,...,...,...,...
123837,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0
123843,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0
123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0
123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,NaN,99201.0


In [ ]:
#make column indicating salary available dont drop just say salary info isnt available
df = df.fillna(0)
df['salary_available'] = df['normalized_salary'].apply(lambda x: 'yes' if x > 0 else 'no')

In [ ]:
df

,company_name,title,description,location,formatted_work_type,formatted_experience_level,work_type,normalized_salary,zip_code,salary_available
70,Revature,Entry Level Oracle Financial Technology Consul...,About RevatureRevature is one of the largest a...,"East Chicago, IN",Full-time,Entry level,FULL_TIME,0.0,46312.0,no
84,Galerie Candy and Gifts,Quality Assurance Manager,Galerie is seeking an experienced Quality Assu...,"Hebron, KY",Full-time,Mid-Senior level,FULL_TIME,0.0,41048.0,no
85,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...","Foster City, CA",Contract,Mid-Senior level,CONTRACT,135200.0,96125.0,yes
101,ActOne Group,Administrative Assistant - CONCUR,Global Financial Services firm is seeking an e...,"New York, NY",Full-time,Associate,FULL_TIME,82500.0,10001.0,yes
102,Aston Carter,Seasonal Office Administrator,Seasonal Office Admin\n\nResponsibilities\n\nW...,"Dayton, OR",Contract,Entry level,CONTRACT,0.0,97114.0,no
...,...,...,...,...,...,...,...,...,...,...
123837,"TalentBurst, an Inc 5000 company",Contract Administrator,"Position: Clinical Contracts Analyst, Req#: 63...","Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes
123843,"TalentBurst, an Inc 5000 company",Quality Engineer,Position: Quality Engineer I (Complaint Invest...,"Irvine, CA",Contract,Mid-Senior level,CONTRACT,83200.0,92602.0,yes
123844,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,"Walnut Creek, CA",Full-time,Mid-Senior level,FULL_TIME,157500.0,94595.0,yes
123846,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,"Spokane, WA",Full-time,Mid-Senior level,FULL_TIME,0.0,99201.0,no


In [ ]:
df.to_csv('filtered_jobpostings.csv')

In [ ]:
df.to_csv('filtered_jobposting_zip.gz', compression='gzip')

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

df = pd.read_csv('filtered_jobpostings.csv')

table = pa.Table.from_pandas(df)
pq.write_table(table, 'filtered_jobpostings.parquet')

In [ ]:
df.to_json('jobpostings.json')